# Lab 1, Exercise 4
# Perform Input and Output Operations


In [ ]:
# This script will create an "Employee Table" with randomized employee names and hire dates and export to a CSV file.
import names, numpy as np
rows = 10000
id = np.array(range(1,10001))
lastname = np.array([''.join(names.get_last_name()) for _ in range(rows)])
firstname = np.array([''.join(names.get_first_name()) for _ in range(rows)])


In [ ]:
# Verify the id, firstname and lastname arrays.  Wait a minute before trying to print the arrays.
print(id)
print(lastname)
print(firstname)


In [ ]:
# Configure and verify date columns
import datetime, random, pandas as pd
nowdate = datetime.date.today()
hiredate = np.array([nowdate - datetime.timedelta(days=(random.randint(30,180))) for _ in range(rows)])
hiretime = np.array([str(random.randint(7,20)) + ':' + str(random.randint(0,59)) + ':00' for _ in range(rows)])
print(hiredate)
print(hiretime)


In [ ]:
# Merge all arrays into a single dataframe
inputzip = zip(id,lastname,firstname,hiredate,hiretime)
inputlist = list(zip(id,lastname,firstname,hiredate,hiretime))
df = pd.DataFrame(inputlist)
print(df)


In [ ]:
# Export data to a csv file
print(df.head(10))
df.to_csv('input.csv',index=False,header=["ID","LastName","FirstName","HireDate","HireTime"])


In [ ]:
# Read the CSV file using the csv module
import csv
with open('input.csv') as csvfile:
    inputcsv = csv.reader(csvfile, delimiter=',')
    for i in inputcsv:
        print(i)
        

In [ ]:
# Visualize stock data using matplotlib
# Choose any three stocks and configure plt and date options
import os, pandas as pd, pandas_datareader as dr, datetime,matplotlib.pyplot as plt
fig = plt.figure()
now = datetime.datetime.now()
begindate = now - datetime.timedelta(days=365)
datestring = datetime.datetime.today().strftime("%Y%m%d%H%M")
stock1 = 'MSFT' 
stock2 = 'ORCL' 
stock3 = 'SAP' 


In [ ]:
# Download stock data and export to CSV files
stockprice1 = dr.DataReader(stock1,"yahoo",begindate,now)   
stockprice2 = dr.DataReader(stock2,"yahoo",begindate,now)  
stockprice3 = dr.DataReader(stock3,"yahoo",begindate,now)  
file1 = datestring+'_'+stock1
file2 = datestring+'_'+stock2
file3 = datestring+'_'+stock3
stockprice1.to_csv(file1+'.csv',encoding='utf-8')
stockprice2.to_csv(file2+'.csv',encoding='utf-8')
stockprice3.to_csv(file3+'.csv',encoding='utf-8')


In [ ]:
# Create lists from the csvs to be used by Matplotlib
x1 = pd.read_csv(file1+'.csv',sep=',').Date  
x1 = [datetime.datetime.strptime(dates,'%Y-%m-%d').date() for dates in x1]
x2 = pd.read_csv(file2+'.csv',sep=',').Date 
x2 = [datetime.datetime.strptime(dates,'%Y-%m-%d').date() for dates in x2]
x3 = pd.read_csv(file3+'.csv',sep=',').Date
x3 = [datetime.datetime.strptime(dates,'%Y-%m-%d').date() for dates in x3]
y1 = pd.read_csv(file1+'.csv',sep=',').Close
y2 = pd.read_csv(file2+'.csv',sep=',').Close
y3 = pd.read_csv(file3+'.csv',sep=',').Close


In [ ]:
# Create Chart
plt.title('Stock Prices of Largest Software Companies')
plt.xlabel('Dates')
plt.ylabel('Stock Price')
# plt.gcf().autofmt_xdate(%m)
# plt.bar(x1,y1,label=stock1)
plt.plot(x1,y1,label=stock1)
plt.plot(x2,y2,label=stock2)
plt.plot(x3,y3,label=stock3)
plt.grid()
plt.legend()


In [ ]:
# Save and Display Chart
fig.savefig(datestring+'_'+stock1+'_'+stock2+'_'+stock3+'.pdf')
fig.savefig(datestring+'_'+stock1+'_'+stock2+'_'+stock3+'.png')
plt.show()


In [ ]:
# This code is an extension of what was done in the previous exercise (You may also view the C:\Labfiles\CreateRGSA.py).
# In addition to creating a storage account, you will now be copying files to a blob and file share that are accessible over the Internet
# Run this code from Azure Cloud Shell
import datetime, os
from azure.common.client_factory import get_client_from_cli_profile
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.storage import StorageManagementClient
from azure.storage.common import CloudStorageAccount
from azure.storage.file import FileService
from azure.storage.blob import PublicAccess
resource_client = get_client_from_cli_profile(ResourceManagementClient)
storage_client = get_client_from_cli_profile(StorageManagementClient)
nameprefix = 'np' + (datetime.datetime.now()).strftime('%H%M%S')
resourcegroupname = nameprefix + 'rg'
storageaccountname = nameprefix + 'sa'
location = 'eastus'
sharename = '55264a'

# You may include the csv files created in the previous cells
os.system('echo "This is a test." > input.csv')
filename = 'input.csv'

# Create the Resource Group and Storage Account
resource_group_params = {'location':location}
resource_client.resource_groups.create_or_update(resourcegroupname, resource_group_params)
storageaccount = storage_client.storage_accounts.create(resourcegroupname, storageaccountname, {'location':location,'kind':'storage','sku':{'name':'standard_ragrs'}})
storageaccount.wait()

# Create Container and Share
sak = storage_client.storage_accounts.list_keys(resourcegroupname, storageaccountname)
storageaccountkey = sak.keys[0].value
cloudstorage_client =  CloudStorageAccount(storageaccountname,storageaccountkey)
blob_service = cloudstorage_client.create_block_blob_service()
blob_service.create_container(sharename,public_access=PublicAccess.Container)
file_service = FileService(account_name=storageaccountname, account_key=storageaccountkey)
file_service.create_share(sharename)

# Copy Setup Files to Container and Share
blob_service.create_blob_from_path(sharename,filename,filename,)
file_service.create_file_from_path(sharename,'',filename,filename,)

# DO NOT DELETE THIS RESOURCE GROUP OR ITS RESOURCES until you complete project4.
# delete_async_operation = resource_client.resource_groups.delete(resourcegroupname)
# delete_async_operation.wait()